In [1]:
import numpy as np
import pandas as pd
import rasterio
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm
from cartopy.feature import NaturalEarthFeature
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy
import pymannkendall as mk

In [2]:
lat = np.array([49.9166666666664 - i * 0.0416666666667 for i in range(357)])
lon = np.array([-105.0416666666507 + i * 0.0416666666667 for i in range(722)])
Lon, Lat = np.meshgrid(lon, lat)

am = rasterio.open('PRISM_tmin_stable_4kmD2_19810101_bil.bil')
a = am.read()[0, :357, 479:1201]
mask = np.where(a>-1000, 1, 0)

tmin81 = np.load('./prism/mw_tmin/1981_tmin.npz')['tmin']
t, y, x = tmin81.shape
y, x

(357, 722)

In [3]:
states = ['MI', 'ND', 'SD', 'NE', 'KS', 'OK', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH', 'KY', 'WV', 'PA', 'AR', 'TN', 'NY', 'MD', 'VA', 'NC']
mdmask = {states[ii]:np.load(f'./var/{states[ii]}_mask.npy') for ii in range(22)}

NWmask = np.where(np.logical_or.reduce((mdmask['ND'], mdmask['SD'], mdmask['NE'])), 1, 0)
SWmask = np.where(np.logical_or.reduce((mdmask['KS'], mdmask['OK'], mdmask['AR'])), 1, 0)
NCmask = np.where(np.logical_or.reduce((mdmask['MN'], mdmask['IA'], mdmask['WI'], mdmask['MI'])), 1, 0)
Cmask = np.where(np.logical_or.reduce((mdmask['MO'], mdmask['IL'], mdmask['IN'], mdmask['OH'], mdmask['KY'], mdmask['TN'], mdmask['WV'])), 1, 0)
NEmask = np.where(np.logical_or.reduce((mdmask['NY'], mdmask['PA'], mdmask['MD'])), 1, 0)
SEmask = np.where(np.logical_or(mdmask['VA'], mdmask['NC']), 1, 0)

In [4]:
data = [np.where(NWmask, 1, np.nan), np.where(SWmask, 2, np.nan), np.where(NCmask, 3, np.nan), np.where(Cmask, 4, np.nan), 
        np.where(NEmask, 5, np.nan), np.where(SEmask, 6, np.nan)]

fig = plt.figure(figsize=(8, 8))
extent = [-105, -75, 34, 49]
ax = plt.axes(projection=ccrs.AlbersEqualArea(np.mean(extent[:2]), np.mean(extent[2:])))
ax.set_extent(extent)
states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
ax.add_feature(states, linewidth=.3, edgecolor="black")
ax.add_feature(cartopy.feature.BORDERS, lw=.3, linestyle=':')
ax.add_feature(cartopy.feature.COASTLINE, lw=.3, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black', facecolor='white', lw=.3)

cmap_name = '6color'
colors= ['magenta','darkorchid', 'orange', 'pink', 'gold', 'red']
cmap = LinearSegmentedColormap.from_list(cmap_name, colors, N=6)
levels = MaxNLocator(nbins=6).tick_values(1, 7)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)

for di in range(6):
    pc = plt.pcolormesh(Lon, Lat, data[di], cmap=cmap, norm=norm, transform=ccrs.PlateCarree())

plt.title('Subregions across Midwest and Central US', fontsize=13, fontweight='bold')

fig.subplots_adjust(bottom=0, top=0.95, left=0.05, right=0.9)
cb_ax = fig.add_axes([0.905, 0.2, 0.02, 0.55])
cbar = fig.colorbar(pc, cax=cb_ax, ticks=np.arange(1.5, 7))
region = ['Northern Rockies and Plains', 'South', 'Upper Midwest', 'Ohio Valley', 'Northeast', 'Southeast']
cbar.ax.set_yticklabels(region, fontweight='bold')
plt.savefig('./plot/Composite/Region_Definition.png', bbox_inches='tight')
plt.close()

In [34]:
region = [NEmask, SEmask, NCmask, Cmask, NWmask, SWmask]

MonStart = [59, 90, 120]
MonStartLeap = [60, 91, 121]
MonDays = [31, 30, 31]
March = np.zeros((39, 6, 31))
April = np.zeros((39, 6, 30))
May = np.zeros((39, 6, 31))
count = [March, April, May]

for yl in range(39):
    tmin = np.load(f'./prism/mw_tmin/{1981+yl}_tmin.npz')['tmin']
    tmin = np.where(tmin>-100, tmin, np.nan)
    if (1981+yl)%4:
        MonS = MonStart
    else:
        MonS = MonStartLeap
    for mi in range(3):
        for si in range(6):
            count[mi][yl, si, :] = np.array([np.where(np.where(region[si], tmin[MonS[mi]+di, :, :], np.nan)<0, 1, 0).sum() / region[si].sum() for di in range(MonDays[mi])])

np.save('./var/MarchCountRegion', count[0])
np.save('./var/AprilCountRegion', count[1])
np.save('./var/MayCountRegion', count[2])

<ipython-input-34-57ea5094e126>:20: RuntimeWarning: invalid value encountered in less
  count[mi][yl, si, :] = np.array([np.where(np.where(region[si], tmin[MonS[mi]+di, :, :], np.nan)<0, 1, 0).sum() / region[si].sum() for di in range(MonDays[mi])])


In [9]:
March = np.load('./var/MarchCountRegion.npy').sum(axis=2).mean(axis=0)
April = np.load('./var/AprilCountRegion.npy').sum(axis=2).mean(axis=0)
May = np.load('./var/MayCountRegion.npy').sum(axis=2).mean(axis=0)
data = [March, April, May, March+April+May]
region = ['Northern Rockies and Plains', 'South', 'Upper Midwest', 'Ohio Valley', 'Northeast', 'Southeast']
mon = ['March', 'April', 'May', 'Total']

table = pd.DataFrame(np.array(data))
table = table.rename(columns={i:region[i] for i in range(6)}, index={i:mon[i] for i in range(4)})
table.round(decimals=3)

,Northern Rockies and Plains,South,Upper Midwest,Ohio Valley,Northeast,Southeast
March,22.641,12.173,26.135,16.387,26.724,14.154
April,10.060,2.699,14.416,4.600,14.884,3.920
May,1.312,0.135,2.580,0.163,2.639,0.175
Total,34.013,15.007,43.131,21.150,44.248,18.248


In [3]:
def color_PositiveNegative(s):
    if s[0] == '-':
        color = 'blue'
        if s[-1] == '*':
            color = 'deepskyblue'
    else:
        color = 'red'
        if s[-1] == '*':
            color = 'magenta'
    return 'color: %s' % color

In [10]:
March = np.load('./var/MarchCountRegion.npy').sum(axis=2)
April = np.load('./var/AprilCountRegion.npy').sum(axis=2)
May = np.load('./var/MayCountRegion.npy').sum(axis=2)
data = [March, April, May, March+April+May]
region = ['Northern Rockies and Plains', 'South', 'Upper Midwest', 'Ohio Valley', 'Northeast', 'Southeast']
mon = ['March', 'April', 'May', 'Total']

slope = np.zeros((4, 6))
pvalue = np.zeros((4, 6))

# X = np.linspace(1, 39, 39)
# for mi in range(4):
#     for si in range(6):
#         r = stats.linregress(X[:], data[mi][:, si])
#         slope[mi, si] = r.slope
#         pvalue[mi, si] = r.pvalue

def trend2num(trend):
    if trend == 'decreasing':
        return -1
    elif trend == 'increasing':
        return 1
    elif trend == 'no trend':
        return 0      
trend = np.zeros((4, 6))
for mi in range(4):
    for si in range(6):
        r = mk.original_test(data[mi][:, si])
        slope[mi, si] = r.slope
        pvalue[mi, si] = r.p
        trend[mi, si] = trend2num(r.trend)

table = [[[] for i in range(4)] for j in range(6)]

for si in range(6):
    for mi in range(4):
        if pvalue[mi, si] < .1:
            table[si][mi] = f'{(slope[mi, si]):.3f}*'
        else:
            table[si][mi] = f'{(slope[mi, si]):.3f}'
table = pd.DataFrame(table).T
table = table.rename(columns={i:region[i] for i in range(6)}, index={i:mon[i] for i in range(4)})
# table = table.round(decimals=2)
table.style.applymap(color_PositiveNegative)

,Northern Rockies and Plains,South,Upper Midwest,Ohio Valley,Northeast,Southeast
March,0.027,-0.001,0.004,-0.031,-0.006,-0.004
April,-0.012,-0.037*,-0.030,-0.060,-0.041,-0.013
May,-0.031*,-0.002,-0.016,-0.001,0.005,0.000
Total,-0.065,-0.065,-0.016,-0.113*,-0.019,-0.003


In [16]:
March = np.load('./var/MarchCountRegion.npy').sum(axis=2)
April = np.load('./var/AprilCountRegion.npy').sum(axis=2)
May = np.load('./var/MayCountRegion.npy').sum(axis=2)
data = [March, April, May, March+April+May]
region = ['Northern Rockies and Plains', 'South', 'Upper Midwest', 'Ohio Valley', 'Northeast', 'Southeast']
mon = ['March', 'April', 'May', 'Total']
X = np.arange(1981, 2020)
fig, axs = plt.subplots(3, 2, figsize=(15, 6))
for ri in range(6):
    plt.subplot(3,2,ri+1)
    plt.plot(X, data[0][:, ri], 'b-o', lw=1, ms=3)
    plt.plot(X, data[1][:, ri], 'g-o', lw=1, ms=3)
    plt.plot(X, data[2][:, ri], 'y-o', lw=1, ms=3)
    plt.plot(X, data[3][:, ri], 'm-o', lw=1, ms=3)
    plt.xticks(np.arange(1981, 2019, 3), [str(X[i]) for i in range(0, 39, 3)])
    plt.ylim(0, 60)
    plt.yticks(np.arange(0, 60, 10))
    plt.text(1981, 55, region[ri], fontsize='large', fontweight='bold')
    plt.grid()
    if ri==1:
        plt.legend(['March', 'April', 'May', 'Total'])
plt.suptitle('Time Series of Freezing Area Percentage', fontsize='large', fontweight='bold')
fig.subplots_adjust(bottom=0, top=0.9, left=0.1, right=0.9, wspace=0.2, hspace=0.01)
plt.savefig('./plot/Composite/FAPTimeSeriesByRegion.png', bbox_inches='tight')
# plt.show()

In [22]:
indi = ['nino34.csv', 'nao.csv', 'pna.csv', 'pdo.csv']
mn = [str(i) for i in range(4, 12)]
# indices_season = []
indices_mon = []
for ii in range(4):
    df = pd.read_csv(f'./index/{indi[ii]}')
#     temp = []
#     for s in range(3):
#         temp.append((np.array(df[mn[3*s]][1:]) + np.array(df[mn[3*s+1]][1:]) + np.array(df[mn[3*s+2]][1:])) / 3.0)
#     w = [(df['12'][k] + df['1'][k+1] + df['2'][k+1])/ 3 for k in range(39)]
#     temp.append(np.array(w))
#     indices_season.append(np.array(temp))
    
    indices_mon.append(np.array([np.array(df['3'][1:]), np.array(df['4'][1:]), np.array(df['5'][1:])]))
# indices_season = np.array(indices_season)
indices_mon = np.array(indices_mon)

In [13]:
X = np.arange(1981, 2020)
yl, yr = [-2, -4, -3, -3], [2, 4, 3, 3]
indna = ['Nino 3.4', 'NAO', 'PNA', 'PDO']
fig, axs = plt.subplots(4, 1, figsize=(12, 8))
for ii in range(4):
    ax = plt.subplot(4,1,ii+1)
    plt.plot(X, indices_mon[ii, 0, :], 'b-o', lw=1, ms=3)
    plt.plot(X, indices_mon[ii, 1, :], 'g-o', lw=1, ms=3)
    plt.plot(X, indices_mon[ii, 2, :], 'y-o', lw=1, ms=3)
    plt.plot(X, indices_mon[ii, :, :].mean(axis=0), 'r-o', lw=1, ms=3)
    plt.xticks(np.arange(1981, 2019, 3), [str(X[i]) for i in range(0, 39, 3)])
    plt.ylim(yl[ii], yr[ii])
    plt.text(0.06, 0.9, indna[ii], fontsize='large', fontweight='bold', ha='center', va='center', transform=ax.transAxes)
    plt.grid()
    plt.legend(['March', 'April', 'May', '3-Month Average'], frameon=False, loc='upper right', ncol=4)
plt.suptitle('Time Series of Teleconnection Indices', fontsize='large', fontweight='bold')
fig.subplots_adjust(bottom=0, top=0.9, left=0.1, right=0.9)
plt.savefig('./plot/Composite/TeleIndTimeSeries.png', bbox_inches='tight')

In [23]:
March = np.load('./var/MarchCountRegion.npy').sum(axis=2)
April = np.load('./var/AprilCountRegion.npy').sum(axis=2)
May = np.load('./var/MayCountRegion.npy').sum(axis=2)
data = [March, April, May, March+April+May]
region = ['Northern Rockies and Plains', 'South', 'Upper Midwest', 'Ohio Valley', 'Northeast', 'Southeast']
mon = ['March', 'April', 'May', 'Total']

slope = np.zeros((4, 4, 6))
pvalue = np.zeros((4, 4, 6))
for ii in range(4):
    for mi in range(4):
        if mi-3:
            index = indices_mon[ii, mi, :]
        else:
            index = indices_mon[ii, :, :].mean(axis=0)
        for si in range(6):
            r = stats.linregress(index, data[mi][:, si])
            slope[ii, mi, si] = r.rvalue
            pvalue[ii, mi, si] = r.pvalue

IndName = ['Nino34', 'NAO', 'PNA', 'PDO']
MonName = [' March', 'April', 'May', 'Total']
table = [[[] for i in range(4)] for j in range(6)]
ii = 3

for si in range(6):
    for mi in range(4):
        if pvalue[ii, mi, si] < .05:
            table[si][mi] = f'{(slope[ii, mi, si]):.3f}**'
        elif pvalue[ii, mi, si] < .1:
            table[si][mi] = f'{(slope[ii, mi, si]):.3f}*'
        else:
            table[si][mi] = f'{(slope[ii, mi, si]):.3f}'
table = pd.DataFrame(table).T
table = table.rename(columns={i:region[i] for i in range(6)}, index={i:MonName[i] for i in range(4)})
print(f'Freezing Days related to {IndName[ii]}')
table.style.applymap(color_PositiveNegative)

Freezing Days related to PDO


,Northern Rockies and Plains,South,Upper Midwest,Ohio Valley,Northeast,Southeast
March,0.086,0.090,0.021,0.074,-0.023,0.014
April,0.037,0.232,0.019,0.114,-0.048,-0.039
May,0.363**,0.277*,0.401**,0.361**,0.119,0.108
Total,0.172,0.206,0.095,0.125,-0.043,-0.006


<font size=4 color=blue>Lag Regression

In [28]:
indi = ['nino34.csv', 'nao.csv', 'pna.csv', 'pdo.csv']
mn = [str(i) for i in range(4, 12)]
lag = 3
indices_mon = []
for ii in range(4): 
    df = pd.read_csv(f'./index/{indi[ii]}')
#     indices_mon.append(np.array([np.array(df[str(3-lag)][1:]), np.array(df[str(4-lag)][1:]), np.array(df[str(5-lag)][1:])]))
    indices_mon.append(np.array([np.array(df['12'][:-1]), np.array(df[str(4-lag)][1:]), np.array(df[str(5-lag)][1:])]))
indices_mon = np.array(indices_mon)

In [29]:
March = np.load('./var/MarchCountRegion.npy').sum(axis=2)
April = np.load('./var/AprilCountRegion.npy').sum(axis=2)
May = np.load('./var/MayCountRegion.npy').sum(axis=2)
data = [March, April, May, March+April+May]
region = ['Northern Rockies and Plains', 'South', 'Upper Midwest', 'Ohio Valley', 'Northeast', 'Southeast']
mon = ['March', 'April', 'May', 'Total']

slope = np.zeros((4, 4, 6))
pvalue = np.zeros((4, 4, 6))
for ii in range(4):
    for mi in range(4):
        if mi-3:
            index = indices_mon[ii, mi, :]
        else:
            index = indices_mon[ii, :, :].mean(axis=0)
        for si in range(6):
            r = stats.linregress(index, data[mi][:, si])
            slope[ii, mi, si] = r.rvalue
            pvalue[ii, mi, si] = r.pvalue

IndName = ['Nino34', 'NAO', 'PNA', 'PDO']
MonName = [' March', 'April', 'May', 'Total']
table = [[[] for i in range(4)] for j in range(6)]
ii = 3

for si in range(6):
    for mi in range(4):
        if pvalue[ii, mi, si] < .05:
            table[si][mi] = f'{(slope[ii, mi, si]):.3f}**'
        elif pvalue[ii, mi, si] < .1:
            table[si][mi] = f'{(slope[ii, mi, si]):.3f}*'
        else:
            table[si][mi] = f'{(slope[ii, mi, si]):.3f}'
table = pd.DataFrame(table).T
table = table.rename(columns={i:region[i] for i in range(6)}, index={i:MonName[i] for i in range(4)})
print(f'Freezing Days related to {IndName[ii]} with {lag}-month Lagging')
table.style.applymap(color_PositiveNegative)

Freezing Days related to PDO with 3-month Lagging


,Northern Rockies and Plains,South,Upper Midwest,Ohio Valley,Northeast,Southeast
March,0.330**,0.188,0.230,0.186,0.226,0.076
April,-0.014,0.105,-0.189,0.092,-0.214,-0.036
May,0.201,-0.002,-0.062,0.043,-0.289*,-0.312*
Total,0.221,0.220,-0.032,0.167,-0.136,-0.014
